Kelompok 11

1.   21.11.3839 M Asngari T
2.   21.11.3849 M Yuzrijal
3.   21.11.3878 Ummi Muyasaroh
4.   21.11.3862 Maghfiroh Izzah M


Deteksi Wajah menggunakan CNN

In [ ]:
import os
import numpy as np
from keras.utils.np_utils import to_categorical
import cv2
import imageio

In [ ]:
images = []
labels = []

In [ ]:
DIRECTORY = "/MyDrive/Colab Notebooks/utkface/UTKFace"
for image in os.listdir(DIRECTORY):
  path = DIRECTORY+image
  gender = path.split('/')[-1].split('_')[1]
  face = imageio.imread(path)
  face = cv2.resize(face, (85,85))
  images.append(face)
  labels.append(int(gender))

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("X : ", len(images))
print("Y : ", len(labels))

In [ ]:
X = np.array(images)
Y = np.array(labels)

X = X.astype('float32')

In [ ]:
X[0].shape

In [ ]:
X_data = np.squeeze(X)
X_data.shape

In [ ]:
X_data /= 255

In [ ]:
Y_data = to_categorical(Y, num_classes=2)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2)

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(X_test.shape)

In [ ]:
print(y_test.shape)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(85, 85, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.fit(X_train,
         y_train,
         batch_size=64,
         epochs=15,
         validation_data=(X_test, y_test),)

In [ ]:
# Mengevaluasi model pada set pengujian
score = model.evaluate(X_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test Accuracy:', score[1])

In [ ]:
labels = ["Laki-laki", "Wanita"] # index 0 = Laki-laki, index 1 = Wanita

print("Laki-laki = ", "0", "\nWanita = ", "1")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
y_hat = model.predict(X_test)

# Plot a random sample of 10 test image, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(X_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(X_test[index]))
    predict_index = np.argmax(y_hat[index])
    true_index = np.argmax(y_test[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(labels[predict_index],
                                 labels[true_index]),
                color=("green" if predict_index == true_index else "red"))

plt.show()